# Εισαγωγή scikit-learn και dataset στο notebook

Αρχικά κάνουμε upgrade στις βιβλοθήκες που θα χρειαστούμε, αν δεν έχουμε ήδη κάνει. Στο Colab θα χρεαστεί πιθανότατα να κάνουμε restart το runtime.

In [29]:
!pip install --upgrade pip
!pip install scikit-learn --upgrade 
!pip install numpy --upgrade 
!pip install --upgrade matplotlib 
!pip install --upgrade pandas

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.

![UCI ML Logo](http://archive.ics.uci.edu/ml/assets/logo.gif "UCI Machine Learning Repository")

To [UCI ML Repository](http://archive.ics.uci.edu/ml/index.php) είναι το διασημότερο αποθετήριο datasets για Machine Learning. Το dataset με το οποίο θα δουλέψουμε είναι το [Dermatology Database](http://archive.ics.uci.edu/ml/datasets/Dermatology). Το dataset περιλαμβάνει διάφορες πληροφορίες σχετικά με δερματολογικές παθήσεις, συγκεκριμένα ESD (Erythemato-Squamous Diseases) καθώς και ετικέτες για κάθε δείγμα (sample), αν αυτό αντιστοιχεί σε μία από τις έξι κατηγορίες ESD. Το σύνολο δεδομένων έχει 365 δείγματα και περιλαμβάνει 34 χαρακτηριστικά (attributes) για κάθε δείγμα, είτε κλινικά  π.χ. βαθμός εξανθήματος, φαγούρα, οικογενειακό ιστορικό είτε ιστοπαθολογικά όπως άκράτεια μελανίνης, εξωκυττάρωση, παρουσία ηωσινόφιλων. Θα χρησιμοποιήσουμε αυτό το dataset και τα χαρακτηριστικά για να προβλέψουμε σε ποια κατηγορία δερματολογικής πάθησης υπάγεται κάθε δείγμα.


In [126]:
import pandas as pd
import numpy as np

def conv(val, default = -1):
    try:
        return int(val)
    except:
        return default

def get_average(values):
    # Remove the values that are not numbers
    existent_values = [int(i) for i in values if i != -1]

    return int(np.average(existent_values))

def load_codon_usage():
    # TO DO: read csv and extract data (pandas)
    # Prepare a dictionary to hold the key value pairs
    output = {'DESCR':"codon Usage Database by Kaggle"}

    # Read CSV and replace the uknown Age values with the average Age.
    dataset = pd.read_csv("codon_usage.csv", low_memory=False, converters={'UUU': conv, 'UUC':conv})
    dataset = dataset.replace(-1, get_average(dataset["UUU"].values))
    dataset = dataset.replace(-1, get_average(dataset["UUC"].values))
    dataset.set_index("SpeciesID")

    # Cannot process string values in MLP. 
    # We have the ID as index se we can drop the whole column
    dataset = dataset.drop(columns='SpeciesName')
    
    # Outlier value. Whole column is float and this is String
    dataset = dataset.replace(['12;I', '-', 'non-B hepatitis virus'], 'NaN')

    # Extract the values
    target_names, counts = np.unique(dataset["Kingdom"], return_counts=True) 
    target = dataset["Kingdom"].values
    data = dataset.iloc[:,1:].values
    feature_names = dataset.columns[1:]

    # Add them to the dictionary
    output['data'] = data
    output['target_names'] = target_names
    output['target'] = target
    output['feature_names'] = feature_names

    return output

# Load dataset
data = load_codon_usage()


Η μεταβλητή "data" είναι ένα αντικείμενο Python που δουλεύει σαν dictionary.

| **key**              | **value**                                             | **type**     | **size**   |
| :------------------- | :---------------------------------------------------- | :----------- | :--------- |
| **'DESCR'**          | 'Breast Cancer Wisconsin (Diagnostic) Database...'    | str          |  1         | 
| **'data'**           | [[1.799, 1.038, 1.228,...]...]                        | float array  |  (569,30)  | 
| **'feature_names'**  | ['mean radius', 'mean texture', ...]                  | str array    | (30,)      |
| **'target'**         | [0, 0, 0, ..., 0, 0, 1]                               | int array    | (569,)     |
| **'target_names'**   | ['malignant', 'benign']                               | str array    | (2,)       |

Τα σημαντικά κλειδιά του λεξικού είναι οι ονομασίες των κατηγοριών εξόδου (target_names), οι κατηγορίες (ή κλάσεις ή ετικέτες) εξόδου (target), τα ονόματα των χαρακτηριστικών (feature_names) και τέλος τα ίδια τα χαρακτηριστικά (data). Στην πράξη χρειαζόμαστε μόνο τα χαρακτηριστικά (features) και τις ετικέτες τους (labels). Οι ονομασίες μας πληροφορούν για τη φυσική ερμηνεία των χαρακτηριστικών.

Δημιουργούμε νέες μεταβλητές για κάθε σημαντικό σύνολο πληροφορίας του dataset:

In [127]:
# Organize our data
label_names = data["target_names"]
labels = data["target"] 
feature_names = data["feature_names"]
features = data["data"]

In [128]:
# Our labels - possible diseases
print(label_names)

['arc' 'bct' 'inv' 'mam' 'phg' 'plm' 'pln' 'pri' 'rod' 'vrl' 'vrt']


In [129]:
# Classes of all dermatology labels where 1:psoriasis incremented to 6:pityriasis rubra pilaris
print(labels)
# Labels are an one dimensional array
print(labels.shape)

unique, counts = np.unique(labels, return_counts=True)
print("frequencies:", counts)

['vrl' 'vrl' 'vrl' ... 'pri' 'pri' 'pri']
(13028,)
frequencies: [ 126 2920 1345  572  220   18 2523  180  215 2832 2077]


In [130]:
# Names of the features
print(feature_names)

Index(['DNAtype', 'SpeciesID', 'Ncodons', 'UUU', 'UUC', 'UUA', 'UUG', 'CUU',
       'CUC', 'CUA', 'CUG', 'AUU', 'AUC', 'AUA', 'AUG', 'GUU', 'GUC', 'GUA',
       'GUG', 'GCU', 'GCC', 'GCA', 'GCG', 'CCU', 'CCC', 'CCA', 'CCG', 'UGG',
       'GGU', 'GGC', 'GGA', 'GGG', 'UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC',
       'ACU', 'ACC', 'ACA', 'ACG', 'UAU', 'UAC', 'CAA', 'CAG', 'AAU', 'AAC',
       'UGU', 'UGC', 'CAU', 'CAC', 'AAA', 'AAG', 'CGU', 'CGC', 'CGA', 'CGG',
       'AGA', 'AGG', 'GAU', 'GAC', 'GAA', 'GAG', 'UAA', 'UAG', 'UGA'],
      dtype='object')


In [131]:
# Name of the first feature
print(feature_names[0])

DNAtype


In [132]:
# οι διαστάσεις όλων των χαρακτηριστικών
print(features.shape)
print(features[5063])


(13028, 67)
[0.00000e+00 3.53569e+05 1.69800e+03 0.00000e+00 0.00000e+00 2.12000e-02
 2.35600e-02 1.17800e-02 1.29600e-02 1.06000e-02 1.29600e-02 4.71000e-03
 6.94900e-02 2.12000e-02 2.12000e-02 4.71000e-03 1.76700e-02 9.42000e-03
 1.53100e-02 8.24000e-03 2.94500e-02 7.07000e-03 1.88500e-02 2.36000e-03
 2.82700e-02 5.89000e-03 4.71000e-03 5.89000e-03 3.29800e-02 1.88500e-02
 1.17800e-02 2.94500e-02 4.71000e-03 1.06000e-02 1.18000e-03 1.17800e-02
 4.71000e-03 1.88500e-02 1.17800e-02 2.59100e-02 3.53000e-03 1.53100e-02
 2.36000e-03 2.12000e-02 2.23800e-02 2.23800e-02 8.24000e-03 3.88700e-02
 8.24000e-03 3.65100e-02 5.89000e-03 1.17800e-02 1.76700e-02 1.29600e-02
 1.88500e-02 1.53100e-02 2.47300e-02 3.06200e-02 1.18000e-03 7.07000e-03
 1.18000e-03 0.00000e+00 2.94500e-02 2.35600e-02 4.47600e-02 2.47300e-02
 1.18000e-03]


Στο συγκεκριμένο παράδειγμα λοιπόν, τo πρώτο δείγμα μας είναι κακοήθες με ακτίνα του κυττάρου  1.79900000e+01.

Έχουμε φορτώσει το Dermatology Dataset, θα προχωρήσουμε στο να δημιουργήσουμε απλούς ταξινομητές για να προβλέπουμε σε ποιά από τις έξι κατηγορίες ανήκουν τα δείγματά μας.

# Train set, test set & αξιολόγηση (ορθότητα) ταξινομητών

Η αξιολόγηση των ταξινομητών γίνεται πάντα σε δεδομένα που δεν έχουν δει κατά την εκπαίδευση έτσι ώστε να αξιολογήσουμε τη δυνατότητα γενίκευσής τους. Συνεπώς, πρωτού φτιάξουμε το μοντέλο κάθε ταξινομητή χωρίζουμε τα δεδομένα μας τυχαία σε ένα σύνολο εκπαίδευσης (train set) και ένα σύνολο ελέγχου (test set).
Παρακάτω βλέπουμε ένα παράδειγμα διαχωρισμού του dataset με αναλογία 60-40:

![data split](https://i.ibb.co/x3V1FQ4/train-test.png)

Χρησιμοποιούμε το train set για να εκτιμούμε και να βελτιώνουμε το μοντέλο του ταξινομητή κατά την ανάπτυξή του. Δεν επιτρέπεται σε κανένα σημείο η χρήση των δεδομένων test για την εκπαίδευση του ταξινομητή.

![training](https://i.ibb.co/MMWjVkn/training.png)

Χρησιμοποιούμε μετά το test set για να αξιολογήσουμε στατιστικά την απόδοση του μοντέλου μας.

![testing](https://i.ibb.co/JsSDLpX/testing.png)

Το sklearn έχει τη συνάρτηση train_test_split() που ανακατεύει τυχαία τα δείγματα και τα διαχωρίζει σε train και test με βάση κάποιο ποσοστό που θα της δώσουμε.

In [133]:
from sklearn.model_selection import train_test_split

# Split our data
train, test, train_labels, test_labels = train_test_split(features, labels, test_size=0.3)

Θα δοκιμάσουμε πρώτα κάποιες πολύ απλές τακτικές ταξινόμησης. Η κλάση DummyClassifier δέχεται μια παράμετρο που καθορίζει την τακτική της ταξινόμησης ως εξής:
* “uniform”: προβλέπει τυχαία και ομοιόμορφα.
* “constant”: προβλέπει πάντα μία κατηγορία που τη διαλέγει ο χρήστης.
* “most_frequent”: προβλέπει πάντα την πιο συχνή κατηγορία στο training set.
* “stratified”: κάνει προβλέψεις διατηρώντας την κατανομή των κλάσεων στο training set.

In [134]:
from sklearn.neural_network import MLPClassifier
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report

clf = MLPClassifier(solver='adam', alpha=1e-5,
                    hidden_layer_sizes=(5,), random_state=1, )
clf.fit(train, train_labels)
preds = clf.predict(test)
print(classification_report(test_labels, preds))

              precision    recall  f1-score   support

         arc       0.00      0.00      0.00        27
         bct       0.22      1.00      0.36       857
         inv       0.00      0.00      0.00       402
         mam       0.00      0.00      0.00       205
         phg       0.00      0.00      0.00        68
         plm       0.00      0.00      0.00         5
         pln       0.00      0.00      0.00       746
         pri       0.00      0.00      0.00        52
         rod       0.00      0.00      0.00        58
         vrl       0.00      0.00      0.00       859
         vrt       0.00      0.00      0.00       630

    accuracy                           0.22      3909
   macro avg       0.02      0.09      0.03      3909
weighted avg       0.05      0.22      0.08      3909



Παρατηρούμε ότι αν τρέξουμε το πρoηγούμενο κελί διαδοχικές φορές, το δεύτερο accuracy αλλάζει γιατί καλούμε εκ νέου τον ταξινομητή να κάνει (τυχαίες) προβλέψεις.

Ας αποθηκεύσουμε την ορθότητα όλων των dummy classifiers σε ένα λεξικό και να την τυπώσουμε από την καλύτερη στη χειρότερη